In [290]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['generic_outliers_data.csv', 'delivery_related.csv', 'session_related.csv']


In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV

In [2]:
session_df= pd.read_csv('session_related.csv')
#sessionData.head()

In [3]:
delivery_df= pd.read_csv('generic_outliers_data.csv')
#genericData.head()

In [4]:
data_df = pd.read_csv('delivery_related.csv')
#deliveryData.head()

In [6]:
firstmerge = pd.merge(delivery_df,session_df,on='OrderId')

KeyError: 'OrderId'

In [296]:
MergedDataset = pd.merge(firstmerge,genericData,on='CustId')
#MergedDataset.head()

In [297]:
IndependentAttributes = pd.DataFrame()

In [298]:
CustId = MergedDataset['CustId']
OrderId = MergedDataset['OrderId']
EmailId = MergedDataset['EmailId']
MobileNo = MergedDataset['MobileNo']
MacAddress = MergedDataset['MacAddress']
AvgPurchase = MergedDataset['AvgPurchase']
City = MergedDataset['City']
IsValidGeo = MergedDataset['IsValidGeo']
IsValidAddress = MergedDataset['IsValidAddress']
IsDeliveryRejected = MergedDataset['IsDeliveryRejected']
ReplacementDate = MergedDataset['ReplacementDate']
IsOneTimeUseProduct = MergedDataset['IsOneTimeUseProduct']
Session_Pincode = MergedDataset['Session_Pincode']
DeliveryDate = MergedDataset['DeliveryDate']
OrderDate = MergedDataset['OrderDate']
Fraud = MergedDataset['Fraud']

In [299]:
IndependentAttributes['CustId'] = CustId
IndependentAttributes['OrderId'] = OrderId
IndependentAttributes['EmailId'] = EmailId
IndependentAttributes['MobileNo'] = MobileNo
IndependentAttributes['MacAddress'] = MacAddress
IndependentAttributes['Session_Pincode'] = Session_Pincode
IndependentAttributes['AvgPurchase'] = AvgPurchase
IndependentAttributes['City'] = City
IndependentAttributes['OrderDate'] = OrderDate
IndependentAttributes['DeliveryDate'] = DeliveryDate

In [300]:
df1 = pd.DataFrame(IndependentAttributes['OrderDate'])
df1['DeliveryDate'] = IndependentAttributes['DeliveryDate']
df1['OrderDate'] = pd.to_datetime(df1['OrderDate'], format='%d/%m/%Y')
df1['DeliveryDate'] = pd.to_datetime(df1['DeliveryDate'], format='%d/%m/%Y')
DaysDifference = (df1['DeliveryDate'] - df1['OrderDate'])
df1.drop(['DeliveryDate', 'OrderDate'], axis='columns', inplace=True)

IndependentAttributes['DaysDifference'] = DaysDifference

In [301]:
IndependentAttributes['ReplacementDate'] = ReplacementDate
IndependentAttributes['IsDeliveryRejected'] = IsDeliveryRejected
IndependentAttributes['IsOneTimeUseProduct'] = IsOneTimeUseProduct
IndependentAttributes['IsValidAddress'] = IsValidAddress
IndependentAttributes['IsValidGeo'] = IsValidGeo
IndependentAttributes['Fraud'] = Fraud

In [302]:
IndependentAttributes.IsDeliveryRejected.replace(('yes', 'no'), (1, 0), inplace=True)
IndependentAttributes.IsOneTimeUseProduct.replace(('yes', 'no'), (1, 0), inplace=True)
IndependentAttributes.IsValidGeo.replace(('YES', 'NO'), (1, 0), inplace=True)
IndependentAttributes.IsValidAddress.replace(('yes', 'no'), (1, 0), inplace=True)
IndependentAttributes.Fraud.replace(('normal', 'suspicious', 'fraudulent'), (0, 1, 1), inplace=True)

In [303]:
IndependentAttributes.MacAddress.replace(regex=r'-', value='', inplace=True)

In [304]:
IndependentAttributes.DeliveryDate.replace(regex=r'/', value='-', inplace=True)
IndependentAttributes.OrderDate.replace(regex=r'/', value='-', inplace=True)
IndependentAttributes.ReplacementDate.replace(regex=r'/', value='-', inplace=True)

In [305]:
IndependentAttributes['DaysDifference'] = IndependentAttributes.DaysDifference.dt.days
IndependentAttributes.head()

,CustId,OrderId,EmailId,MobileNo,MacAddress,Session_Pincode,AvgPurchase,City,OrderDate,DeliveryDate,DaysDifference,ReplacementDate,IsDeliveryRejected,IsOneTimeUseProduct,IsValidAddress,IsValidGeo,Fraud
0,47019,41601,aQGHuxe@yahoo.com,1867946110,1DBA453FE543,276001,99522,Latur,25-01-2019,28-01-2019,3,01-02-2019,1,0,1,1,0
1,20242,30352,sej94kG@hotmail.com,9272875375,65D1F362F744,505468,3491,Mumbai G.P.O.,06-02-2019,10-02-2019,4,17-02-2019,0,1,0,1,0
2,79619,12328,HTVm_XUBz@gmail.com,3251871005,DF6DBF6C2359,586209,5262,Chhatarpur City,12-01-2019,22-01-2019,10,01-02-2019,1,0,1,0,1
3,79619,80750,HTVm_XUBz@gmail.com,3251871005,DF6DBF6C2359,791102,5262,Chhatarpur City,19-01-2019,26-01-2019,7,31-01-2019,0,1,0,0,0
4,80669,19203,dPiRkMo3i@gmail.com,3152175434,495560A5F52B,250001,4957,Sunamura,27-03-2019,03-04-2019,7,06-04-2019,0,0,1,1,0


In [306]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [307]:
df = pd.DataFrame() 
df = reduce_mem_usage(IndependentAttributes)

Memory usage of dataframe is 13.73 MB
Memory usage after optimization is: 10.89 MB
Decreased by 20.7%


In [308]:
Fraud_txn = df[df['Fraud']== 1]
normal_txn = df[df['Fraud']== 0]

# print("---------------------------")
# print("From the training dataset:")
# print("---------------------------")
# print("  Total Customers : %i"\
#       %(len(df)))
# print("")
# print("  Total Normal transactions  : %i"\
#       %(len(normal_txn)))

# print("  Normal transactions Rate   : %i %% "\
#      % (1.*len(normal_txn)/len(df)*100.0))
# print("-------------------------")

# print("  Fraudulent transactions         : %i"\
#       %(len(Fraud_txn)))

# print("  Fraudulent transactions Rate    : %i %% "\
#      % (1.*len(Fraud_txn)/len(df)*100.0))
# print("-------------------------")

In [309]:
from sklearn.preprocessing import LabelEncoder
categorical = ['IsValidAddress','IsDeliveryRejected','IsOneTimeUseProduct','IsValidGeo','CustId','City']

label_encoder = LabelEncoder()
for col in categorical:
    df[col] = label_encoder.fit_transform(df[col].astype(str))

df=df.iloc[:df.shape[0]]

In [310]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
features = pd.DataFrame(df)
labels = pd.DataFrame(features['Fraud'])
features = features.drop(['Fraud','EmailId','MacAddress','City'], axis=1)
features = features.apply(LabelEncoder().fit_transform)
#features.head()

In [311]:
#labels.head()

In [312]:
# Scaling the Train and Test feature set 
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#features = scaler.fit_transform(features)
features.head()

,CustId,OrderId,MobileNo,Session_Pincode,AvgPurchase,OrderDate,DeliveryDate,DaysDifference,ReplacementDate,IsDeliveryRejected,IsOneTimeUseProduct,IsValidAddress,IsValidGeo
0,26030,41501,6101,134,35167,74,90,0,0,1,0,1,1
1,7189,30252,58249,383,2422,18,34,1,59,0,1,0,1
2,49061,12228,15784,522,4133,35,72,7,0,1,0,1,0
3,49061,80650,15784,787,4133,56,84,4,104,0,1,0,0
4,49784,19103,15101,112,3836,82,8,4,17,0,0,1,1


In [313]:
labels.head()

,Fraud
0,0
1,0
2,1
3,0
4,0


In [314]:
from sklearn.model_selection import train_test_split
train, test, labels_train, labels_test = train_test_split(features, labels, test_size=0.60, random_state = 42)

In [315]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, labels_train, test_size=0.20, random_state = 42)

In [316]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
x_train_std_os,y_train_os = sm.fit_sample(X_train,y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [317]:

from sklearn.ensemble.forest import RandomForestClassifier
rf = RandomForestClassifier(random_state=0).fit(x_train_std_os,y_train_os)
rf_pred = rf.predict(X_val)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [318]:
from sklearn import metrics

In [319]:
###Just gave the output of few algo with smote which classifies both classes and without smote it doesnt classify both classes

In [320]:
temp1 = metrics.f1_score(y_val,rf_pred,average=None)[1]
temp2 = metrics.f1_score(y_val,rf_pred,average=None)[0]

# F1Fraud= temp1*100
# F1Normal= temp2*100


F1Fraud = str(round(temp1, 2)*100)
F1Normal = str(round(temp2, 2)*100)


#print(F1Fraud,F1Normal,sep="\n")

###Just gave the output of few algo with smote which classifies both classes and without smote it doesnt classify both classes

In [321]:
# import json
# data = {"F1 Fraud":F1Fraud,"F1 Normal":F1Normal, "model": "SVM"}
# jstr = json.dumps(data)

# with open('F1.json', 'w') as outfile: 
#     jstr
# import json
# with open('data.json', 'w') as outfile:
#     data = {"F1":metrics.f1_score(y_val,pred,average=None), "model": "SVM"}
#     json.dump(data, outfile)

# data = {"F1":metrics.f1_score(y_val,pred,average=None), "model": "SVM"}    
# import json
# with open('data.txt', 'w') as f:
#     json.dump(data, f, ensure_ascii=False)

import json

data = {
    'f1_fraud': F1Fraud,
    'f1_normal': F1Normal,
    'model': 'SVM'
}


with open("data_file.json", "w") as write_file:
    json.dump(data, write_file)
    
with open("data_file.json", "r") as read_file:
    data = json.load(read_file)
    
print(data)

{'f1_fraud': '26.0', 'f1_normal': '74.0', 'model': 'SVM'}


In [322]:
features.head()

,CustId,OrderId,MobileNo,Session_Pincode,AvgPurchase,OrderDate,DeliveryDate,DaysDifference,ReplacementDate,IsDeliveryRejected,IsOneTimeUseProduct,IsValidAddress,IsValidGeo
0,26030,41501,6101,134,35167,74,90,0,0,1,0,1,1
1,7189,30252,58249,383,2422,18,34,1,59,0,1,0,1
2,49061,12228,15784,522,4133,35,72,7,0,1,0,1,0
3,49061,80650,15784,787,4133,56,84,4,104,0,1,0,0
4,49784,19103,15101,112,3836,82,8,4,17,0,0,1,1


In [323]:
# y_hats2 = model.predict(X)

# features['y_hats'] = y_hats2

rf_pred

array([1, 1, 0, ..., 1, 0, 0], dtype=int8)

In [324]:
rf_pred

array([1, 1, 0, ..., 1, 0, 0], dtype=int8)

In [325]:
res = pd.DataFrame(rf_pred)
results = res.rename(columns={0:'Fraud'})

In [326]:
results.head()

,Fraud
0,1
1,1
2,0
3,1
4,0


In [327]:
final_export = pd.DataFrame()

In [328]:
OrderId = features['OrderId']
#OrderDate = features['OrderDate']
#City = features['City']
AvgPurchase = features['AvgPurchase']
PredictedLabel = results['Fraud']
#EmailId = features['EmailId']


In [329]:
final_export['OrderId'] = OrderId 
final_export['CustId'] = CustId 
final_export['City'] = City 
final_export['PredictedLabel'] = PredictedLabel 
final_export['AvgPurchase'] = AvgPurchase
final_export['EmailId'] = EmailId
final_export['DeliveryDate']=DeliveryDate
final_export['OrderDate'] = OrderDate 


In [330]:
final_export.head()

,OrderId,CustId,City,PredictedLabel,AvgPurchase,EmailId,DeliveryDate,OrderDate
0,41501,47019,Latur,1.0,35167,aQGHuxe@yahoo.com,28/01/2019,25/01/2019
1,30252,20242,Mumbai G.P.O.,1.0,2422,sej94kG@hotmail.com,10/02/2019,06/02/2019
2,12228,79619,Chhatarpur City,0.0,4133,HTVm_XUBz@gmail.com,22/01/2019,12/01/2019
3,80650,79619,Chhatarpur City,1.0,4133,HTVm_XUBz@gmail.com,26/01/2019,19/01/2019
4,19103,80669,Sunamura,0.0,3836,dPiRkMo3i@gmail.com,03/04/2019,27/03/2019


In [289]:
final_export.to_csv(“final_export.csv”, sep=’,’)

SyntaxError: invalid character in identifier (<ipython-input-289-545c3ea48370>, line 1)

In [331]:
export_csv = final_export.to_csv (r'abc.csv', index = None, header=True)